In [2]:
%%capture
%load_ext autoreload
%autoreload 2
%matplotlib inline
# %cd .. 
import sys
sys.path.append("..")
import statnlpbook.util as util
import matplotlib

<!---
Latex Macros
-->
$$
\newcommand{\Xs}{\mathcal{X}}
\newcommand{\Ys}{\mathcal{Y}}
\newcommand{\y}{\mathbf{y}}
\newcommand{\balpha}{\boldsymbol{\alpha}}
\newcommand{\bbeta}{\boldsymbol{\beta}}
\newcommand{\aligns}{\mathbf{a}}
\newcommand{\align}{a}
\newcommand{\source}{\mathbf{s}}
\newcommand{\target}{\mathbf{t}}
\newcommand{\ssource}{s}
\newcommand{\starget}{t}
\newcommand{\repr}{\mathbf{f}}
\newcommand{\repry}{\mathbf{g}}
\newcommand{\x}{\mathbf{x}}
\newcommand{\prob}{p}
\newcommand{\vocab}{V}
\newcommand{\params}{\boldsymbol{\theta}}
\newcommand{\param}{\theta}
\DeclareMathOperator{\perplexity}{PP}
\DeclareMathOperator{\argmax}{argmax}
\DeclareMathOperator{\argmin}{argmin}
\newcommand{\train}{\mathcal{D}}
\newcommand{\counts}[2]{\#_{#1}(#2) }
\newcommand{\length}[1]{\text{length}(#1) }
\newcommand{\indi}{\mathbb{I}}
$$

# Relation Extraction 
Relation extraction (RE) is the task of extracting semantic relations between arguments. Arguments can either be general concepts such as "a company" (ORG), "a person" (PER); or instances of such concepts (e.g. "Microsoft", "Bill Gates"), which are called proper names or named entitites (NEs). An example for a semantic relation would be "founder-of(PER, ORG)". Relation extraction therefore often builds on the task of [named entity recognition](/template/statnlpbook/02_methods/00_structuredprediction).

Relation extraction is relevant for many high-level NLP tasks, such as

* for question answering, where users ask questions such as "Who founded Microsoft?",
* for information retrieval, which often relies on large collections of structured information as background data, and 
* for text and data mining, where larger patterns in relations between concepts are discovered, e.g. temporal patterns about startups

<!-- TODO: Load NYT or other dataset, update link to NER -->

## Relation Extraction as Structured Prediction
We can formalise relation extraction as an instance of [structured prediction](/template/statnlpbook/02_methods/00_structuredprediction) where the input space \\(\Xs\\) are pairs of arguments \\(\Es\\) and supporting texts \\(\Ss\\) those arguments appear in. The output space \\(\Ys\\) is a set of relation labels such as \\(\Ys=\\{ \text{founder-of}, \text{employee-at}, \text{professor-at}, \text{NONE}\\} \\). The goal is to define a model \\(s_{\params}(\x,y)\\) that assigns high *scores* to the label \\(y\\) that fits the arguments and supporting text \\(\x\\), and lower scores otherwise. The model will be parametrized by \\(\params\\), and these parameters we will learn from some training set \\(\train\\) of \\((\x,y)\\) pairs. When we need to classify input  instances \\(\x\\) consisting again of pairs of arguments and supporting texts, we have to solve the maximization problem $\argmax_y s_{\params}(\x,y)$. Note that this frames relation extraction as a multi-class classification problem (Exercise: how could RE be formalised to predict multiple labels for each input instance and how would the example below have to be adapted for that?)


## Relation Extraction Example
Before we take a closer look at relation extraction methods, let us consider a concrete example.
<!-- Expand on this -->


## Pattern-Based Extraction
<!-- Expand on this -->


## Bootstrapping
<!-- Expand on this -->


## Supervised Relation Extraction
<!-- Expand on this -->


## Distant Supervision
<!-- Expand on this -->


## Universal Schema
<!-- Expand on this -->


## Background
Jurafky & Martin etc